In [3]:
import os
os.chdir(os.path.dirname(os.getcwd()))
from helpers import *
from dataset_helpers import *
import torch
import json
import cv2

from tqdm import tqdm
from PIL import Image

### Yolo5 for V3C dataset

In [3]:
data = dataset(dataset_name='V3C')
data.get_file_name()
print("Number of images: ", len(data.image_names))

Loading all image names ...
Number of images:  2508110


In [23]:
video_name_path = osp.join(DATASET_MASTER_PATH, 'indices')
with open(osp.join(video_name_path, 'video_indices.txt'), 'r') as file:
    video_indices = file.read().splitlines()

for video in video_indices:
    curr_path = osp.join(DATASET_MASTER_PATH, 'object_detection/labels', video)
    if not osp.isdir(curr_path):
        os.mkdir(curr_path)

In [1]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='object_detection/yolov5_models/yolov5m.pt')

for item in tqdm(data.image_names):
    concepts = convert_to_concepts(item)
    label_curr_path = osp.join(DATASET_MASTER_PATH, 'object_detection/labels', concepts['video'])
    image_curr_path = osp.join(DATASET_MASTER_PATH, 'object_detection/images', concepts['video'])
    detection = model(item) 
    temp = detection.save(image_curr_path)
    bbox = detection.pandas().xyxy[0]
    save_df_to_json(bbox, osp.join(label_curr_path, '{0}.json'.format(concepts['filename'][:-4])))
    